Library

In [1]:
import string
import re
from collections import OrderedDict
from tabulate import tabulate
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

Membaca dataset

In [10]:
df = pd.read_csv("news_data.csv")
df = df[['article_text']].dropna().head(20)
text = [[f"Doc {i+1}", row['article_text']] for i, row in df.iterrows()]


Setup Library

In [4]:
stopfactory = StopWordRemoverFactory()
stopword = stopfactory.create_stop_word_remover()
stemfactory = StemmerFactory()
stemmer = stemfactory.create_stemmer()

Preprocessing Function

In [12]:
def preprocess_text(text):
    title, content = text

    # Clean the content
    clean = "".join([c for c in content if c not in string.punctuation]).lower()
    clean = re.sub(r'\d+', '', clean)  # Remove numbers

    # Tokenize the content
    tokens = clean.split()

    #Stopword removal
    filtered = [word for word in tokens if word not in stopwords.words('indonesian')]

    # Stemming
    stemmed = [stemmer.stem(word) for word in filtered]

    # Terms
    terms = list(OrderedDict.fromkeys(stemmed))
    return [title, clean, tokens, filtered, stemmed, terms]

    

Preprocessing Output in Table

In [ ]:
headers = ["Title", "Clean", "Tokens", "Filtered", "Stemmed", "Terms"]
for idx, doc in enumerate(text):
    hasil = preprocess_text(doc)
    print(f"Document {idx+1}:")
    print(tabulate([hasil], headers=headers, tablefmt="grid"))
    print('=' * 100)
    print("\n")

Save as .csv

In [ ]:
semua_hasil = []
for idx, doc in enumerate(text):
    hasil = preprocess_text(doc)
    semua_hasil.append(hasil)

hasil_df = pd.DataFrame(semua_hasil, columns=headers)
hasil_df.to_csv("preprocesse d_news_data.csv", index=False)